In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mocpy import MOC
from astropy.io import votable
import mod as mod

### Task 1 : Extract table in VOTable format.

In [ ]:
limit = 100 # the table contain>100K lines
source1 = "II/7A/catalog"
url = f"https://vizier.cds.unistra.fr/viz-bin/votable?-source={source1}&-out.max=10000"
vtable = votable.parse(url)

# get information
for info in vtable.iter_info():
    print(f"{info.name:<20} {info.value}")

Here, a regular table from astropy

In [ ]:
from astropy.table import Table

limit = 100000 # the table contain>100K lines

source = "II/7A/catalog"
url = f"https://vizier.cds.unistra.fr/viz-bin/votable?-source={source}&-out.max={limit}"

table = Table.read(url, table_id="II/7A/catalog")
print(table[:5])

### Task 2 : Build an astropy table with an added HEALPix column in the order that you choose (eg: order 10) You can use the astropy-healpix library to compute the HEALPix number from ra,dec.

In [ ]:
from astropy_healpix import HEALPix
from astropy.coordinates import SkyCoord, ICRS, Angle
from astropy import units as u

order = 4 # We choose any order
nside = 2**order
table = Table.read(url, table_id="II/7A/catalog")

# Compute the HEALPix index and add it as a new column

hp = HEALPix(nside=nside, order = 'nested', frame = ICRS())
coord = SkyCoord(table["_RA"], table["_DE"], frame= 'icrs', unit = 'deg')
list_ipix = hp.skycoord_to_healpix(coord)

table['Healpix'] = list_ipix
print("Lenth of the ipix list is = ", len(list_ipix))
print(table[:5])

### Task 3: Create a simple HEALPix pixelation from the HEALPix column (so a kind of MOC but in a unique order). Your code must contains a text serialization. Example of HEALPix pixelation serialization: 6/17407 18090 18773 19456

In [ ]:
from astropy_healpix import HEALPix

order = 3 #We can put any order here.
nside = 2**order

hp = HEALPix(nside=nside, order = 'nested', frame = ICRS())
coord = SkyCoord(table["_RA"], table["_DE"], frame= 'icrs', unit = 'deg')
list_ipix = hp.skycoord_to_healpix(coord)
unique_list = (np.unique(np.sort(np.abs(list_ipix))))
string_list = " ".join(map(str, unique_list))

with open('Serialised_text.txt', 'w') as f:
    f.write(f"{order}/")
    f.writelines(''.join(string_list))
    
print(f"{order}/", string_list)

### Task 4: Extend the code to build a real MOC (with HEALPix in different order) and apply it to the HEALPix column. Your code must contains a text serialization. Todo that, you can (but you can follow an other algorithm) have a recursive approach that manages cells in a structure (made of dictionaries, lists, objects) and functions/methods that manages node operations (like to add or delete a node). Then you iterate the HEALPix values and feed one by one the MOC tree.

In [1]:
import mod as mod 

#I import my module here

##### Here, I print the print the order and the ideal number of ipix cells it should contain.

In [ ]:
order = 1
mod.details(order) 

##### Now, I build a function which gives me the list of ipix cells of the current order. It also generates a list, called "main_list" which will be used in subsequent functions like "My_MOC" which, for example, is defined to generate the MOC.

In [ ]:
order = 1 #Insert any order
a = mod.current_order(order)
print(f"{order}/", a)

##### Now, I build a function which gives me the list of ipix cells of the higher order to the current order.

In [ ]:
order = 1 #Insert any order
main_list = mod.current_order(order) 
mod.increase_order(order, main_list) #This function gives the list of ipix cells of the next order.
 
print(f"{order+1}/", mod.increase_order(order, main_list))

##### Now I build the final function, which will construct the desired MOC Tree

In [ ]:
order=6
main_list = mod.current_order(order)  
print(mod.My_MOC(order,main_list))

###  Task 5 : Illustrate the result in a Jupyter notebook with plots (matplotlib) or in Aladin (module ipyaladin). In the notebook, use your serialization and mocpy (especially for functions like plots or moc-operation only available in mocpy).

In [ ]:
from mocpy import MOC

order = 8
main_list = mod.current_order(order)

moc = MOC.from_str(mod.My_MOC(order, main_list)) # Here, I use my serialisation

fig = plt.figure(111, figsize=(15, 10))
wcs = moc.wcs(fig, coordsys="icrs", rotation=Angle(0, u.degree), projection="AIT")
ax = fig.add_subplot(1, 1, 1, projection=wcs)
moc.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="green")
moc.border(ax=ax, wcs=wcs, alpha=0.5, color="black")
plt.title(f"MOC of {source} using serialisation, starting at order = {order}", fontsize=16)
plt.grid(color="black", linestyle="dotted")
plt.savefig(r"MOC_using_serialisation.png")
plt.show()


### Note: to test, you can use the mocpy library and check if the results are similar. You can also plot the result using astropy (see the mocpy doc.). The mocpy library is able to check some inconsistencies like overlapping elements.

In [ ]:
order = 8
nside = 2**order

# Here, I use vizier data to confirm if my function is right
moc = MOC.from_vizier_table(table_id = f"{source}", nside=nside) #source is defined in the beginning of the notebook

fig = plt.figure(111, figsize=(15, 10))
wcs = moc.wcs(fig, coordsys="icrs", rotation=Angle(0, u.degree), projection="AIT")
ax = fig.add_subplot(1, 1, 1, projection=wcs)
moc.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="green")
moc.border(ax=ax, wcs=wcs, alpha=0.5, color="black")
plt.title(f"MOC of {source} using VizieR data, starting at order = {order}", fontsize=16)
plt.grid(color="black", linestyle="dotted")
plt.savefig(r"MOC_using_VizieR.png")
plt.show()
